In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openke
from openke.config import Trainer, Tester
from openke.module.loss import SigmoidLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

from openke_model import RotAtte

In [3]:
import torch
import numpy as np

In [4]:
negative_rate = 10

In [5]:
train_dataloader = TrainDataLoader(
	in_path = "./benchmarks/FB15K237/", 
	batch_size = 10000,
	threads = 1,
	sampling_mode = "cross", 
	bern_flag = 0, 
	filter_flag = 1, 
	neg_ent = negative_rate,
	neg_rel = 0
)

facts = TrainDataLoader(
	in_path = "./benchmarks/FB15K237/", 
	batch_size = train_dataloader.get_triple_tot(),
	threads = 1,
	sampling_mode = "normal", 
	bern_flag = 0, 
	filter_flag = 1, 
	neg_ent = 0,
	neg_rel = 0
)

h, t, r, _, _ = [f for f in facts][0].values()
h = torch.Tensor(h).to(torch.long)
t = torch.Tensor(t).to(torch.long)
r = torch.Tensor(r).to(torch.long)

facts = torch.stack((h, r, t)).cuda().t()

In [6]:
test_dataloader = TestDataLoader("./benchmarks/FB15K237/", "link")

In [7]:
rotatte = RotAtte(
    n_ent = train_dataloader.get_ent_tot(),
    n_rel = train_dataloader.get_rel_tot(),
    in_dim = 100, 
    out_dim = 100,
    facts = facts,
    negative_rate=negative_rate,
)

In [8]:
model = NegativeSampling(
	model = rotatte, 
	loss = SigmoidLoss(adv_temperature = 2),
	batch_size = train_dataloader.get_batch_size(), 
	regul_rate = 0.0
)

In [9]:
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 100, alpha = 1e-2, use_gpu = True, opt_method = "sgd")

In [ ]:
trainer.run()
rotatte.save_embeddings()
# rotatte.save_checkpoint("rotatte.ckpt")

  0%|          | 0/100 [00:00<?, ?it/s]

Finish initializing...


Epoch 21 | loss: 18.554899:  22%|██▏       | 22/100 [02:20<08:19,  6.41s/it]

In [ ]:
# rotatte.load_checkpoint("rotatte.ckpt")
tester = Tester(model = rotatte, data_loader = test_dataloader, use_gpu = True)
result = tester.run_link_prediction(type_constrain = False)

In [ ]:
result